<a href="https://colab.research.google.com/github/ErickMBarreto/TechChallenge3/blob/main/TechChallenge3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fiap Tech Challenge 3

**Apresentação**

Este notebook foi elaborado para o Tech Challenge, uma atividade que consolida e utiliza os conceitos aprendidos durante o curso. A meta principal é executar o fine-tuning de um foundation model
 (neste projeto, o LLaMA-3 8B, em sua versão quantizada de 4 bits com a biblioteca Unsloth), empregando o dataset AmazonTitles-1.3MM.

As etapas do processo são:

1º **Realizar o pré-processamento do dataset** a partir do arquivo trn.json, que inclui os campos title (nome do produto) e content (descrição).

2º **Estruturar os dados em prompts no estilo Alpaca**, organizados com Instruction, Input e Response, para viabilizar o treinamento supervisionado.

3º **Aplicar o fine-tuning ao modelo base utilizando a metodologia LoRA** (Low-Rank Adaptation), que permite otimizar o treinamento mesmo com recursos computacionais restritos.

4º **Analisar a performance do modelo antes e depois do ajuste**, medindo sua habilidade de criar descrições lógicas com base nos títulos dos produtos.

5º **Preparar o modelo ajustado para a inferência**, de modo que ele possa responder às consultas do usuário, fornecendo a descrição apropriada conforme o conhecimento adquirido no fine-tuning.

Conectando ao Google Drive

Estabelecendo a conexão com o Google Drive, onde os arquivos necessários para o treinamento estão armazenados.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Instalando a biblioteca para acelerar o treino**

Instala também o transformers e o dataset

In [4]:
!pip install "unsloth[colab-new]" -U
!pip install --no--deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 23.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    

Modelo Llama 3

In [6]:
from unsloth import FastLanguageModel

model_id = "unsloth/llama-3-8b-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

/tmp/ipython-input-1312953121.py:1: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.1: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Função de Inferência para Geração de Respostas

Este trecho define a função run_chat, responsável por realizar a inferência do modelo a partir de um prompt fornecido pelo usuário:

1. Tokenização → O texto de entrada (prompt) é convertido em tensores pelo tokenizer e enviado para a GPU (cuda).

2. Geração de texto → O modelo recebe os tokens e gera uma continuação da sequência, limitada a max_new_tokens=64.

3. Decodificação → A saída numérica é convertida de volta para texto legível com tokenizer.batch_decode.

Essa função encapsula o fluxo completo de interação com o modelo, permitindo que o usuário teste diferentes prompts e visualize as respostas geradas de forma simples e direta.

In [7]:
def run_chat(prompt, tokenizer, model):
  inputs = tokenizer([prompt], return_tensors='pt').to('cuda')
  outputs = model.generate(
      **inputs,
      max_new_tokens=64,
      use_cache = True
  )
  return tokenizer.batch_decode(outputs)


Inferência com Prompt Personalizado

Este trecho executa a função run_chat, enviando ao modelo um prompt que solicita detalhes sobre o livro Harry Potter and the Philosopher's Stone.

O texto é processado pelo tokenizer e passado ao modelo já treinado.

O modelo gera uma resposta baseada em seu conhecimento e no fine-tuning realizado.

A saída é retornada ao usuário em linguagem natural, simulando uma consulta real sobre um livro técnico de desenvolvimento de software.

Esse exemplo serve como validação prática do funcionamento do pipeline de inferência criado no notebook.

In [28]:
run_chat("Could you provide details about the book Clean Architeture: A Craftsman's Guide to Software Structure and Design?", tokenizer=tokenizer, model=model)

['<|begin_of_text|>Could you provide details about the book Clean Architeture: A Craftsman\'s Guide to Software Structure and Design? I am interested in the book and would like to know more about it. Thank you.\nI am looking for a book titled "Clean Architecture: A Craftsman\'s Guide to Software Structure and Design". I have not been able to find it in any book store, or online. Can you provide details about the book and']

Pré-processamento e Geração do Dataset para Fine-Tuning

Esse código faz o pré-processamento de dados para criar um conjunto de dados em formato JSON, ideal para o treinamento e ajuste fino (fine-tuning) de modelos de linguagem. Ele pega um arquivo de entrada chamado trn.json e o transforma em um novo arquivo, data.json, que servirá como o dataset final.

In [10]:
import json

#Arquivo de entrada(Dados brutos em JSON)
raw_file_path = '/content/drive/MyDrive/TechC3/trnF.json'

#Config
INSTRUCTION = "Describe book as accurately as possible."
QUESTION = "Could you give me a description of the book '{}'?"
MAX_RECORDS = 1000000
counter = 0

final_data = []

#Leitura e processamento do arquivo bruto
with open(raw_file_path, 'r', encoding = 'utf-8') as file:
  for row in file:
    try:
      json_parsed = json.loads(row)
      title = json_parsed.get('title')
      content = json_parsed.get('content')

      if not title or not content:
        continue

      input = f'Question: {QUESTION.format(title)}'

      #Gera múltiplos exemplos (variações repetidas)
      for i in range(1, 100):
        final_data.append(
            {
                "instruction": INSTRUCTION,
                "input": input,
                "output": content
            }
        )

        counter = len(final_data)

        #Exibe progresso a cada 50.000 registros
        if counter % 50000 == 0:
          print(f"counter: {counter}")

        #Interrompe se atinger o limite máximo
        if counter >= MAX_RECORDS:
          break
    except Exception as e:
      print(row)
      raise


#Arquivo de saída (JSON formatado)
format_data_file_path = '/content/drive/MyDrive/TechC3/data.json'
with open(format_data_file_path, 'w') as f:
  json.dump(final_data, f, indent=2)


A saída de streaming foi truncada nas últimas 5000 linhas.
counter: 1200
counter: 1400
counter: 1600
counter: 1800
counter: 2000
counter: 2200
counter: 2400
counter: 2600
counter: 2800
counter: 3000
counter: 3200
counter: 3400
counter: 3600
counter: 3800
counter: 4000
counter: 4200
counter: 4400
counter: 4600
counter: 4800
counter: 5000
counter: 5200
counter: 5400
counter: 5600
counter: 5800
counter: 6000
counter: 6200
counter: 6400
counter: 6600
counter: 6800
counter: 7000
counter: 7200
counter: 7400
counter: 7600
counter: 7800
counter: 8000
counter: 8200
counter: 8400
counter: 8600
counter: 8800
counter: 9000
counter: 9200
counter: 9400
counter: 9600
counter: 9800
counter: 10000
counter: 10200
counter: 10400
counter: 10600
counter: 10800
counter: 11000
counter: 11200
counter: 11400
counter: 11600
counter: 11800
counter: 12000
counter: 12200
counter: 12400
counter: 12600
counter: 12800
counter: 13000
counter: 13200
counter: 13400
counter: 13600
counter: 13800
counter: 14000
counter: 1

Formatação do Dataset para Fine_Tuning

Neste trecho está sendo preparado o dataset bruto para ser utilizado no treinamento supervisionado de modelos de linguagem.

In [11]:
#Importa a função load_dataset da biblioteca datasets (Hugging Face)
#Usada para carregar o dataset JSON.
from datasets import load_dataset

#Define o template (prompt) no estilo Alpaca.
#Esse formato organiza as intruções, entradas e saídas de forma padronizada.
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}

"""

#Define o token de fim de sequência do tokenizer (vai ser adicionado ao final do texto).
EOS_TOKEN = tokenizer.eos_token

#Função para formatar os exemplos do dataset no padrão Alpaca.
def formatting_prompts_func(examples):
  #Extrai as colunas relevantes do dataset (cada uma é uma lista).
  instructions = examples['instruction']
  inputs = examples['input']
  outputs = examples['output']
  texts = []

  #Junta os calores de instruction, input e output em um unico texto, aplicando o templete definido acima.
  for instruction, input, output in zip(instructions, inputs, outputs):
    text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
    texts.append(text)

  #Retorna no formato esperando pelo Hugging Face Datasets (coluna "text").
  return {'text': texts}

#Carrega o dataset salvo no Google Drive (aquele "final_data.json" gerado antes).
dataset = load_dataset(
    "json",
    data_files='/content/drive/MyDrive/TechC3/data.json',
    split='train'
)

#Aplica a função de formatação para transformar o dataset no formato pronto para treinamento (instrução + input + resposta).
dataset = dataset.map(formatting_prompts_func, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1001148 [00:00<?, ? examples/s]

O resultado é um dataset unificado e pronto para ser usado no treinamento de modelos de linguagem adaptados via LoRA ou SFT(Supervised Fine-Tuning)


Configuração do Fine-Tuning com LoRA e SFTTrainer

Este trecho do código adapta um modelo de linguagem utilizando a técnica LoRA (Low-Rank Adaptation) para reduzir o custo de treinamento e otimizar o uso de memória. Em seguida, cria um objeto SFTTrainer (Supervised Fine-Tuning Trainer), responsável por executar o processo de ajuste fino supervisionado no dataset formatado previamente.

In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

#Adapta o modelo para LoRA (Low-Rank Adaptation)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout = 0.05,
    bias = "none"
)


#Define o treinador para fine-tuning supervisionado
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,        #Tamanho do batch por GPU
        gradient_accumulation_steps = 4,        #Acumula gradientes antes de atualizar os pesos
        warmup_steps = 5,                       #Passos de aquecimento do LR
        max_steps = 100,                        #Número total de steps de treino
        learning_rate = 2e-4,                   #Taxa de aprendizado inicial
        fp16 = not is_bfloat16_supported(),     #Usa FP16 se não houver suporte ao bfloat16
        bf16 = is_bfloat16_supported(),         #Usa bfloat16 se suportado
        logging_steps = 1,                      #Frequência de logs
        optim = "adamw_8bit",                   #Otimizador eficiente em memória
        weight_decay = 0.01,                    #Penalização para evitar overfitting
        lr_scheduler_type = "linear",           #Schedule linear para LR
        seed = 7043,                            #Semente para reprodutibilidade
        output_dir = "outputs",                 #Onde salvar checkpoints/resultados
    ),
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.10.1 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/1001148 [00:00<?, ? examples/s]

**Saída**

Este Trecho salva no Google Drive o modelo ajustado com LoRA e o tokenizer utilizado no treinamento.

In [13]:
model.save_pretrained("/content/drive/MyDrive/TechC3/lora_dataF")
tokenizer.save_pretrained("/content/drive/MyDrive/TechC3/lora_dataF")

('/content/drive/MyDrive/TechC3/lora_dataF/tokenizer_config.json',
 '/content/drive/MyDrive/TechC3/lora_dataF/special_tokens_map.json',
 '/content/drive/MyDrive/TechC3/lora_dataF/tokenizer.json')

**Carregamento do Modelo Base com Adaptador LoRA para inferência**

Este trecho de código carrega o modelo LLaMA-3 em 4 bits utilizando a biblioteca Unsloth, e aplica o adaptador LoRA previamente treinado e salvo no Google Drive.

In [15]:
!ls /content/drive/MyDrive/TechC3/lora_dataF

adapter_config.json	   README.md		    tokenizer_config.json
adapter_model.safetensors  special_tokens_map.json  tokenizer.json


**Resultado: Geração de Texto com Pipeline**

Este trecho de código utiliza a biblioteca Transformers para facilitar a inferência por meio do pipeline de geração de texto.

In [33]:
#Importa a biblioteca de pipeline para facilitar
from transformers import pipeline
import torch

#Instrução e pergunta
INSTRUCTION = "Describe book as accurately as possible."
QUESTION = "Question: Could you give me a description of the book '{}'?"
book_title = "Clean Architeture: A Craftsman's Guide to Software Structure and Design"

#Template no formato Alpaca
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}

"""

#Prepara o prompt final
final_prompt = alpaca_prompt.format(
    INSTRUCTION,
    QUESTION.format(book_title),
    ""
)

#Cria o pipeline de geração de texto
pipe = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    torch_dtype = torch.float16,
    device_map = "auto"
)


#Gera a saída do modelo
outputs = pipe(
    final_prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
)

#Exibe o texto gerado
print(outputs[0]['generated_text'])

Device set to use cuda:0


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Describe book as accurately as possible.

### Input:
Question: Could you give me a description of the book 'Clean Architeture: A Craftsman's Guide to Software Structure and Design'?

### Response:
Clean Architecture: A Craftsman's Guide to Software Structure and Design is a book written by Robert C. Martin, published by Prentice Hall in 2017. It provides a comprehensive guide to software architecture, with an emphasis on creating maintainable, flexible, and efficient systems. The book is divided into three parts: The Philosophy of Clean Architecture, The Principles of Clean Architecture, and The Practice of Clean Architecture. The Philosophy of Clean Architecture covers the underlying principles and values that underlie good software architecture, including separation of concerns, modularity, and simplicity. The 

**Conclusão**

Neste notebook, realizamos todo o fluxo de fine-tuning supervisionado utilizando a técnica LoRA (Low-Rank Adaptation) em um modelo LLaMA-3 quantizado em 4 bits, com o apoio da biblioteca Unsloth para maior eficiência.

As etapas desenvolvidas incluíram:

1. **Configuração do ambiente** e instalação das dependências necessárias.

2. **Pré-processamento e formatação do dataset**, transformando dados brutos no padrão Alpaca (Instruction, Input e Response).

3. **Adaptação do modelo com LoRA**, permitindo treinar de forma eficiente mesmo em recursos computacionais limitados.

4. **Treinamento supervisionado** com parâmetros otimizados para balancear custo computacional e qualidade do ajuste.

5. **Salvamento e recarregamento** do modelo e tokenizer, garantindo reuso sem necessidade de retreinar.

6. **Inferência com pipeline** de geração de texto, validando a capacidade do modelo em responder a prompts personalizados.

Com isso, foi possível compreender e aplicar na prática o processo de ajuste fino de modelos de linguagem para um caso de uso específico (descrição de livros). O pipeline pode ser adaptado para outros domínios de conhecimento, bastando alterar o dataset de entrada e as instruções fornecidas.